# 📎 Appendix – Power Query M Code

**raw_data query**
```M
let
    Source = Web.BrowserContents("https://www.atm.it/it/IlGruppo/personale/Pagine/Adesionescioperi.aspx"),
    #"Extracted Table From Html" = Html.Table(Source, {{"Column1", "TABLE:nth-child(3) > * > TR > TD[rowspan=""2""]:not([colspan]):nth-child(1):nth-last-child(3), TABLE:nth-child(3) > * > TR > TD:not([colspan]):not([rowspan]):nth-child(1):nth-last-child(3)"}, {"Column2", "TABLE:nth-child(3) > * > TR > TD[rowspan=""2""]:not([colspan]):nth-child(1):nth-last-child(3) + TD[rowspan=""2""]:not([colspan]):nth-child(2):nth-last-child(2), TABLE:nth-child(3) > * > TR > TD:not([colspan]):not([rowspan]):nth-child(1):nth-last-child(3) + TD:not([colspan]):not([rowspan]):nth-child(2):nth-last-child(2)"}, {"Column3", "TABLE:nth-child(3) > * > TR > TD[rowspan=""2""]:not([colspan]):nth-child(1):nth-last-child(3) + TD[rowspan=""2""]:not([colspan]):nth-child(2):nth-last-child(2) + TD:not([colspan]):not([rowspan]):nth-child(3):nth-last-child(1), TABLE:nth-child(3) > * > TR > TD:not([colspan]):not([rowspan]):nth-child(1):nth-last-child(1), TABLE:nth-child(3) > * > TR > TD:not([colspan]):not([rowspan]):nth-child(1):nth-last-child(3) + TD:not([colspan]):not([rowspan]):nth-child(2):nth-last-child(2) + TD:not([colspan]):not([rowspan]):nth-child(3):nth-last-child(1)"}}, [RowSelector="TABLE:nth-child(3) > * > TR"]),
    #"Promoted Headers" = Table.PromoteHeaders(#"Extracted Table From Html", [PromoteAllScalars=true]),
    #"Changed Type" = Table.TransformColumnTypes(#"Promoted Headers",{{"Data", type date}, {"Proclamante", type text}, {"Percentuale di adesione", type text}})
in
    #"Changed Type"
    
```

**clean_data query**
---
```M
let
    Source = raw_data,
    #"Renamed Columns" = Table.RenameColumns(Source,{{"Data", "Date"}, {"Proclamante", "Unions"}, {"Percentuale di adesione", "Partecipation"}}),
    #"Removed Top Rows" = Table.Skip(#"Renamed Columns",1),
    #"Replaced Value" = Table.ReplaceValue(#"Removed Top Rows","Sono stati garantiti solo i servizi essenziali comunicati nell'avviso di sciopero.","67%",Replacer.ReplaceText,{"Partecipation"}),
    #"Replaced Value1" = Table.ReplaceValue(#"Replaced Value",".",",",Replacer.ReplaceText,{"Partecipation"}),
    #"Replaced Value2" = Table.ReplaceValue(#"Replaced Value1","%","",Replacer.ReplaceText,{"Partecipation"}),
    #"Changed Type" = Table.TransformColumnTypes(#"Replaced Value2",{{"Partecipation", type number}}),
    #"Rounded Off" = Table.TransformColumns(#"Changed Type",{{"Partecipation", each Number.Round(_, 0), type number}}),
    #"Divided Column" = Table.TransformColumns(#"Rounded Off", {{"Partecipation", each _ / 100, type number}}),
    #"Uppercased Text" = Table.TransformColumns(#"Divided Column",{{"Unions", Text.Upper, type text}}),
    #"Replaced Value3" = Table.ReplaceValue(#"Uppercased Text",".","",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value4" = Table.ReplaceValue(#"Replaced Value3","OS ","",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value5" = Table.ReplaceValue(#"Replaced Value4","OOSS ","",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value6" = Table.ReplaceValue(#"Replaced Value5","SEGRETERIA NAZIONALE ","",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value7" = Table.ReplaceValue(#"Replaced Value6","USB LAVORO PRIVATO#(lf)            FILT CGIL FIT CISL UILT UIL FAISA CISAL UGL","USB LAVORO PRIVATO, FILT CGIL, FIT CISL, UILT UIL, FAISA CISAL, UGL",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value8" = Table.ReplaceValue(#"Replaced Value7","CGIL UIL-CUB E SGB","CGIL, UIL, CUB, SGB",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value9" = Table.ReplaceValue(#"Replaced Value8","AL COBAS UNITAMENTE AD ALTRE AUTONOME E DI BASE (CUB PUBBLICO IMPIEGO, FAO, LAVORATORI METALMECCANICI ORGANIZZATI, SINDACATO GENERALE DI CLASSE, SLAIPROLCOBAS E SINDACATO OPERAI AUTORGANIZZATI)","AL COBAS, CUB PUBBLICO IMPIEGO, FAO, LAVORATORI METALMECCANICI ORGANIZZATI, SINDACATO GENERALE DI CLASSE, SLAIPROLCOBAS, SINDACATO OPERAI AUTORGANIZZATI",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value10" = Table.ReplaceValue(#"Replaced Value9","CONFAIL – FAISA","CONFAIL FAISA",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value11" = Table.ReplaceValue(#"Replaced Value10","1) FILT CGIL, FIT CISL, UILTRASPORTI, FAISA CISAL, UGL (24 ORE)#(lf)            2) CUB TRASPORTI (24 ORE)#(lf)            3) SLM FAST CONFSAL (4 ORE)#(lf)            4) CONFAIL FAISA (4 ORE)","FILT CGIL, FIT CISL, UILTRASPORTI, FAISA CISAL, UGL, CUB TRASPORTI, SLM FAST CONFSAL, CONFAIL FAISA",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value12" = Table.ReplaceValue(#"Replaced Value11",";",",",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value13" = Table.ReplaceValue(#"Replaced Value12","- ",", ",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value14" = Table.ReplaceValue(#"Replaced Value13"," - ",", ",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value15" = Table.ReplaceValue(#"Replaced Value14"," -",", ",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value16" = Table.ReplaceValue(#"Replaced Value15"," E ",", ",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value17" = Table.ReplaceValue(#"Replaced Value16"," CON ADESIONE DI ",", ",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value18" = Table.ReplaceValue(#"Replaced Value17"," – ",", ",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value19" = Table.ReplaceValue(#"Replaced Value18","SINDACATO GENERALE DI BASE","SGB",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value20" = Table.ReplaceValue(#"Replaced Value19","SINDACATO GENERALE DI CLASSE","SGC",Replacer.ReplaceText,{"Unions"}),
    #"Replaced Value21" = Table.ReplaceValue(#"Replaced Value20","-"," ",Replacer.ReplaceText,{"Unions"}),
    #"Trimmed Text" = Table.TransformColumns(#"Replaced Value21",{{"Unions", Text.Trim, type text}}),
    #"Replaced Value22" = Table.ReplaceValue(#"Trimmed Text",",,",",",Replacer.ReplaceText,{"Unions"})
in
    #"Replaced Value22"
```
---


**processed_data query**
---
```M
let
    Source = clean_data,
    #"Added Custom" = Table.AddColumn(Source, "Day", each Date.DayOfWeekName([Date])),
    #"Reordered Columns" = Table.ReorderColumns(#"Added Custom",{"Date", "Day", "Unions", "Partecipation"}),
    #"Added Custom1" = Table.AddColumn(#"Reordered Columns", "Union Count", each List.Count(Text.Split([Unions], ","))),
    #"Changed Type" = Table.TransformColumnTypes(#"Added Custom1",{{"Union Count", Int64.Type}})
in
    #"Changed Type"
```
---


**exploded_data query**
---
```M
let
    Source = processed_data,
    #"Split Column by Delimiter" = Table.ExpandListColumn(Table.TransformColumns(Source, {{"Unions", Splitter.SplitTextByDelimiter(",", QuoteStyle.Csv), let itemType = (type nullable text) meta [Serialized.Text = true] in type {itemType}}}), "Unions"),
    #"Changed Type" = Table.TransformColumnTypes(#"Split Column by Delimiter",{{"Unions", type text}}),
    #"Trimmed Text" = Table.TransformColumns(#"Changed Type",{{"Unions", Text.Trim, type text}}),
    #"Renamed Columns" = Table.RenameColumns(#"Trimmed Text",{{"Unions", "Union"}}),
    #"Added Custom" = Table.AddColumn(#"Renamed Columns", "Main Union", each if Text.Contains([Union], "CGIL") then "CGIL" 
else if Text.Contains([Union], "CISL") then "CISL"
else if Text.Contains([Union], "UIL") then "UIL"
else if Text.Contains([Union], "UGL") then "UGL"
else if Text.Contains([Union], "CUB") then "CUB"
else if Text.Contains([Union], "USB") then "USB"
else if Text.Contains([Union], "CONFAIL") then "CONFAIL"
else if Text.Contains([Union], "CISAL") then "CISAL"
else if Text.Contains([Union], "COBAS") then "COBAS"
else if Text.Contains([Union], "SGB") then "SGB"
else if Text.Contains([Union], "ORSA") then "ORSA"
else "OTHER"),
    #"Reordered Columns" = Table.ReorderColumns(#"Added Custom",{"Date", "Day", "Main Union", "Union", "Partecipation", "Union Count"}),
    #"Changed Type1" = Table.TransformColumnTypes(#"Reordered Columns",{{"Main Union", type text}, {"Day", type text}}),
    #"Trimmed Text1" = Table.TransformColumns(#"Changed Type1",{{"Main Union", Text.Trim, type text}, {"Union", Text.Trim, type text}}),
    #"Replaced Value" = Table.ReplaceValue(#"Trimmed Text1"," ","-",Replacer.ReplaceText,{"Union"})
in
    #"Replaced Value"
```
---